Install dependencies

In [18]:
%pip install langchain
%pip install langchain_community
%pip install torch
%pip install transformers
%pip install sentence-transformers
%pip install datasets
%pip install faiss-cpu
%pip install streamlit
%pip install -U langchain-huggingface

  Obtaining dependency information for langchain-core<0.3.0,>=0.2.38 from https://files.pythonhosted.org/packages/bc/02/2b2cf9550cee1a7ffa42fe60c55e2d0e7d397535609b42562611fb40e78d/langchain_core-0.2.41-py3-none-any.whl.metadata
  Using cached langchain_core-0.2.41-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.41-py3-none-any.whl (397 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.10
    Uninstalling langchain-core-0.3.10:
      Successfully uninstalled langchain-core-0.3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.0 requires langchain-core<0.4,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to

In [20]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [48]:
!pip install datasets

In [51]:
from langchain_community.document_loaders import TextLoader
file_name = "romeo-and-juliet.txt"

# Create a loader instance
loader = TextLoader(file_name) 
# Specify the dataset name and the column containing the content
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"  # or any other column you're interested in

# Create a loader instance
# loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

# Display the first 15 entries
data[:2]

[Document(metadata={'source': 'romeo-and-juliet.txt'}, page_content='\ufeffThe Project Gutenberg eBook of Romeo and Juliet\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: Romeo and Juliet\n\nAuthor: William Shakespeare\n\nRelease date: November 1, 1998 [eBook #1513]\n                Most recently updated: June 19, 2024\n\nLanguage: English\n\nCredits: the PG Shakespeare Team, a team of about twenty Project Gutenberg volunteers\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK ROMEO AND JULIET ***\n\n\n\n\nTHE TRAGEDY OF ROMEO AND JULIET\n\nby William Shakespeare\n\n\n\n\nContents

In [52]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

docs[0]

Document(metadata={'source': 'romeo-and-juliet.txt'}, page_content='\ufeffThe Project Gutenberg eBook of Romeo and Juliet\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: Romeo and Juliet\n\nAuthor: William Shakespeare\n\nRelease date: November 1, 1998 [eBook #1513]\n                Most recently updated: June 19, 2024\n\nLanguage: English\n\nCredits: the PG Shakespeare Team, a team of about twenty Project Gutenberg volunteers\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK ROMEO AND JULIET ***\n\n\n\n\nTHE TRAGEDY OF ROMEO AND JULIET\n\nby William Shakespeare\n\n\n\n\nContents\

In [153]:

# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings()
#     model_name=modelPath,     # Provide the pre-trained model's path
#     model_kwargs=model_kwargs, # Pass the model configuration options
#     encode_kwargs=encode_kwargs # Pass the encoding options
# )

/var/folders/fs/_nfqpbxd2y19j30k267t_lrw0000gn/T/ipykernel_62747/2362150411.py:11: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [154]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.04895174875855446, -0.03986188396811485, -0.02156275510787964]

In [155]:
db = FAISS.from_documents(docs, embeddings)

In [157]:
import torch
# Specify the model name you want to use
model_name = "distilgpt2"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad", torch_dtype=torch.float32)
# question_answerer = pipeline(
#     "text-generation", 
#     model=model_name, 
#     tokenizer=tokenizer,
#     # max_length=1024,
#     # truncation=True,   # Enable truncation
#     # padding=True       # Enable padding
# )

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={
        "temperature": 0.7,
        # "max_new_tokens": 50,  # Use max_new_tokens instead of max_length
        # "max_length": 512,
    },
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [158]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

In [170]:
question = "Which is Romeo's family?"
search_docs = db.similarity_search(question)
# Check if any documents were retrieved
if not search_docs:
    print("No relevant documents found.")
else:
    # Prepare input data for the QA system
    context = " ".join([doc.page_content for doc in search_docs]) 
    # max_context_length = 50  # Set a limit for the context length
    # if len(search_docs) > max_context_length:
    #     search_docs = search_docs[:max_context_length] 
    # Combine question and context
    # Combine question and context
    combined_input = f"{question} {context}"

    # Tokenize the combined input to check its length
    tokenized_input = tokenizer(combined_input, return_tensors='pt')

    # If tokenized input length exceeds max_length, truncate it explicitly
    # if len(tokenized_input['input_ids'][0]) > tokenizer.model_max_length:
    #     # Truncate and decode to maintain coherence
    #     combined_input = tokenizer.decode(tokenized_input['input_ids'][0][:tokenizer.model_max_length], skip_special_tokens=True)

    # Prepare input data
    input_data = {
        "query": combined_input  # Use the truncated combined input
    }

    # print(len(context))
    # print(len(question))
    print(combined_input)

Which is Romeo's family? ROMEO.
Not mad, but bound more than a madman is:
Shut up in prison, kept without my food,
Whipp’d and tormented and—God-den, good fellow.

SERVANT.
God gi’ go-den. I pray, sir, can you read?

ROMEO.
Ay, mine own fortune in my misery.

SERVANT.
Perhaps you have learned it without book.
But I pray, can you read anything you see?

ROMEO.
Ay, If I know the letters and the language.

SERVANT.
Ye say honestly, rest you merry!

ROMEO.
Stay, fellow; I can read.

 [_He reads the letter._]

_Signior Martino and his wife and daughters;
County Anselmo and his beauteous sisters;
The lady widow of Utruvio;
Signior Placentio and his lovely nieces;
Mercutio and his brother Valentine;
Mine uncle Capulet, his wife, and daughters;
My fair niece Rosaline and Livia;
Signior Valentio and his cousin Tybalt;
Lucio and the lively Helena. _


A fair assembly. [_Gives back the paper_] Whither should they come?

SERVANT.
Up.

ROMEO.
Whither to supper?

SERVANT.
To our house.

ROMEO.
Whose

In [171]:
# Prepare input data
# input_data = {
#     "query": combined_input  # Use the truncated combined input
# }
# input_data = {
#     "query": question,
#     "context": context  # Include the retrieved documents as context
# }

# Invoke the QA system with the input data
try:
    result = qa.invoke(input=input_data)
    # Print the result
    # print(result["result"])
except ValueError as e:
    print(f"Error: {e}")
    print(f"Input length: {len(tokenized_input['input_ids'][0])}, Max length: {tokenizer.model_max_length}")



Error: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

ROMEO.
Not mad, but bound more than a madman is:
Shut up in prison, kept without my food,
Whipp’d and tormented and—God-den, good fellow.

SERVANT.
God gi’ go-den. I pray, sir, can you read?

ROMEO.
Ay, mine own fortune in my misery.

SERVANT.
Perhaps you have learned it without book.
But I pray, can you read anything you see?

ROMEO.
Ay, If I know the letters and the language.

SERVANT.
Ye say honestly, rest you merry!

ROMEO.
Stay, fellow; I can read.

 [_He reads the letter._]

_Signior Martino and his wife and daughters;
County Anselmo and his beauteous sisters;
The lady widow of Utruvio;
Signior Placentio and his lovely nieces;
Mercutio and his brother Valentine;
Mine uncle Capulet, his wife, and daughters;
My fair niece Rosaline and Livia;
Signior Valentio and his cousin Tybalt;
Lucio and the lively Helena. _


